In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile"

def initialize_driver():
    """
    Initializes the Selenium WebDriver and maximizes the browser window.
    """
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    """
    Loads the given URL in the WebDriver and waits for the page to load.
    """
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    """
    Scrapes all available bus route links and names on the page.
    """
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    """
    Scrapes bus details for a specific route URL and name.
    """
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load

        try:
            # Click the "View Buses" button if it exists
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load

            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text if i < len(bus_name_elements) else '',
                    "Bus_Type": bus_type_elements[i].text if i < len(bus_type_elements) else '',
                    "Departing_Time": departing_time_elements[i].text if i < len(departing_time_elements) else '',
                    "Duration": duration_elements[i].text if i < len(duration_elements) else '',
                    "Reaching_Time": reaching_time_elements[i].text if i < len(reaching_time_elements) else '',
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                    "Price": price_elements[i].text if i < len(price_elements) else '',
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details

        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# Main function to scrape all pages and details
def scrape_all_pages():
    """
    Scrapes all pages of the website, collects bus details, and saves to a CSV.
    """
    all_bus_details = []  # List to hold all bus details
    driver = initialize_driver()  # Initialize WebDriver

    try:
        for page in range(1, 6):  # Loop through 5 pages
            load_page(driver, URL)
            
            if page > 1:
                # Navigate to the correct pagination tab
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.element_to_be_clickable((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            # Scrape bus routes
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            
            # Scrape details for each route
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                all_bus_details.extend(bus_details)

    except Exception as e:
        print(f"Error occurred: {str(e)}")
    
    finally:
        # Close the driver
        driver.quit()

    # Save details to a CSV
    df = pd.DataFrame(all_bus_details)
    df.to_csv('up_bus_details.csv', index=False)
    print("Data saved to up_bus_details.csv")

# Run the scraper
scrape_all_pages()


Data saved to up_bus_details.csv


In [4]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,UPSRTC - HDD0006,Janrath AC Seater 2+2,18:30,12h 16m,06:46,3.3,INR 1178,30 Seats available
1,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,UPSRTC - KRJ0018,Ordinary Non AC Seater 2+3,18:30,11h 30m,06:00,3.3,INR 796,52 Seats available
2,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,UPSRTC - KRJ0022,Ordinary Non AC Seater 2+3,19:00,12h 00m,07:00,3.6,INR 796,52 Seats available
3,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,,,,,,,,
4,Delhi to Lucknow,https://www.redbus.in/bus-tickets/delhi-to-luc...,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...
519,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - STD0072,Janrath AC Seater 2+2,22:40,07h 05m,05:45,2.5,INR 818,36 Seats available
520,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - KSB0395,Janrath AC Seater 2+2,23:05,06h 40m,05:45,3.3,INR 818,35 Seats available
521,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - LKM0401,Shatabdi AC Seater 2+2,23:15,06h 59m,06:14,3.3,INR 818,35 Seats available
522,Shahjahanpur (Uttar Pradesh) to Delhi,https://www.redbus.in/bus-tickets/shahjahanpur...,UPSRTC - GND0100,Ordinary Non AC Seater 2+3,23:55,08h 05m,08:00,3.3,INR 586,52 Seats available
